In [1]:
import nltk
from pandas import DataFrame, read_csv
from string import punctuation
from pathlib import Path

### Importa arquivo

In [2]:
news_data_path = Path('../arquivos/mongo_export.csv')
news_df = read_csv(news_data_path)

In [3]:
news_df = news_df.drop(columns=['_id'])
news_df.head()

,code,source,url,title,datetime,raw_text
0,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/ceo-do-b...,CEO do Banco do Brasil diz que houve falha de ...,2021-02-12T10:41:35.000Z,André Brandão (Foto: Marcelo Camargo/ Agência ...
1,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/despesas...,Despesas permanecerão sob rigoroso controle ne...,2021-02-12T07:10:00.000Z,André Brandão (Foto: Marcelo Camargo/ Agência ...
2,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/bradesco...,"Bradesco, Itaú, Santander ou Banco do Brasil: ...",2021-02-12T17:41:41.000Z,SÃO PAULO – A temporada de resultados do quart...
3,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/andre-br...,Ações do Banco do Brasil fecham em queda de 5%...,2021-01-13T18:11:02.000Z,"SÃO PAULO – As ações do Banco do Brasil (, R$ ..."
4,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/banco-do...,Banco do Brasil anuncia abertura de 14 novas a...,2021-02-03T13:55:00.000Z,Seguindo o planejamento de reestruturação orga...


### Define funções para o tratamento dos textos

In [4]:
def tokenize(text: str) -> list:
    """ Divide the text into token """
    toknzr = nltk.tokenize.WhitespaceTokenizer()
    return toknzr.tokenize(text)

In [5]:
def remove_pt_stop_words(text: str) -> str:
    """ remove stop words from text in portuguese"""
    stop_words = nltk.corpus.stopwords.words("portuguese")
    no_stop_text = [word for word in text.split() if word not in stop_words]

    return ' '.join(no_stop_text)

In [6]:
def remove_punctuation(text: str) -> str:
    """ Remove punctuation from text """
    punctuation_list = ''.join([punctuation, chr(8211), chr(8220), chr(8221)])  # try regex
    punctuation_list = punctuation_list.replace('-', '')
    no_punct_text =  [word for word in text if word not in punctuation_list]
    no_punct_text = ''.join(no_punct_text)
    return no_punct_text

In [7]:
def stemmize(tokens: list) -> list:
    stemmer = nltk.RSLPStemmer()
    return [stemmer.stem(tok) for tok in tokens]

### Aplicação das funções no DataFrame

In [8]:
news_df['pp_text'] = news_df['raw_text'].apply(remove_punctuation)

In [9]:
news_df['pp_text'] = news_df['pp_text'].apply(remove_pt_stop_words)

### Tokens a serem utilzados no treinamento

In [18]:
news_df['tokens'] = news_df['pp_text'].apply(tokenize)

In [23]:
news_df['tokens'] = news_df['tokens'].apply(stemmize)

In [24]:
news_df['tokens']

0      [andré, brand, fot, marcel, camarg, agênc, bra...
1      [andré, brand, fot, marcel, camarg, agênc, bra...
2      [são, paul, a, tempor, result, quart, trimestr...
3      [são, paul, as, açõ, banc, brasil, r, 3755, -4...
4      [segu, planej, reestrutur, organizac, anunci, ...
                             ...                        
288    [são, paul, oop, lev, qued, nest, sexta-f, 19,...
289    [shutterstocks, paul, oop, qued, nest, quarta-...
290    [shutterstocks, paul, ofech, lev, qued, nest, ...
291    [são, paul, o, ibovesp, futur, abr, qued, nest...
292    [shutterstocks, paul, ofech, est, nest, sexta-...
Name: tokens, Length: 293, dtype: object